In [2]:
import psycopg2
import psycopg2.extras
import configparser as configparser

In [3]:
def parse_ini(section: str) -> dict:
    """
    This function parses ini file for configuration details
    :param section: section to read from ini
    :return: Dictionary of config details
    """
    config = dict()
    parser = configparser.ConfigParser()
    parser.read("database.ini")
    if parser.has_section(section):
        config_items = parser.items(section)
        for item in config_items:
            config[item[0]] = item[1]
    return config

In [4]:
db_config = parse_ini("postgresql")
db_config

{'host': 'localhost',
 'database': 'instacart',
 'user': 'postgres',
 'password': 'postgres',
 'port': '5432'}

In [5]:
def load_entities(db_config: dict, list_of_csvs: list) -> bool:
    """
    This function insert dataframe to an existing entity table
    :param db_config: Database configuration details
    :param list_of_csvs:
    :return: boolean whether process was done or not
    """
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()
    for csv_name, table_name in list_of_csvs:
        try:
            with open(csv_name + '.csv', 'r', encoding='utf-8') as csvfile:
                next(csvfile)  # To skip header row.
                print("Copying from CSV:" + csv_name + " to table:" + table_name + " of DB")
                cursor.copy_from(csvfile, table_name, sep='|', null='\\N')
                conn.commit()
                print("The CSV is loaded to DB")
        except (Exception, psycopg2.DatabaseError) as error:
            print("SQL Exception:" + str(error))
            conn.rollback()
            return False
    cursor.close()
    conn.close()
    return True

In [6]:
list_of_csvs = [("aisles_df", "aisles"), ("departments_df", "departments"), ("random_us_addresses_df", "users"), ("products_df", "products"), ("orders_df", "orders"), ("order_products_prior_df", "order_products")]
list_of_csvs

[('aisles_df', 'aisles'),
 ('departments_df', 'departments'),
 ('random_us_addresses_df', 'users'),
 ('products_df', 'products'),
 ('orders_df', 'orders'),
 ('order_products_prior_df', 'order_products')]

In [ ]:
load_entities(db_config, list_of_csvs)